In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor  
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_excel('Data_Train.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [3]:
df.shape

(10683, 11)

In [4]:
df.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [5]:
df.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [6]:
# Dropping the row which accounts for the 2 NAs
df = df.dropna(axis = 0)

#### Handling similar columns

In [7]:
# Building consistency in the cities comprised in Source and Destination
df['Destination'] = df['Destination'].str.replace('New ','')
df['Destination'] = df['Destination'].str.replace(' ','')
df['Destination'] = df['Destination'].str.replace('Delhi','New Delhi')
df['Source'] = df['Source'].str.replace('Delhi','New Delhi')

In [8]:
# Checking the new status quo with 'Source' and 'Destination'
print(df['Source'].unique())
print(df['Destination'].unique())

['Banglore' 'Kolkata' 'New Delhi' 'Chennai' 'Mumbai']
['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Hyderabad']


In [9]:
# Encoding the cities
df['Source'] = df['Source'].replace({'Banglore': 0, 'Kolkata': 1, 'New Delhi': 2, 'Chennai': 3, 'Mumbai': 4})
df['Destination'] = df['Destination'].replace({'Banglore': 0, 'Kolkata': 1, 'New Delhi': 2, 'Cochin': 5, 'Hyderabad': 6})

#### Handling datetime columns

In [10]:
# Changing to appropriate data types
df['Date_of_Journey'] = df['Date_of_Journey'].astype('datetime64')
df['Dep_Time'] = df['Dep_Time'].astype('datetime64[ns]')
df['Arrival_Time'] = df['Arrival_Time'].astype('datetime64[ns]')

In [11]:
# Extracting the time of departure of the flights
df['Dep_Time'] =  df['Dep_Time'].apply(lambda x: x.time())
# Extracting Arrival Time and Date
df['Date_of_Arrival'] = df['Arrival_Time'].apply(lambda x: x.date())
df['Arrival_Time'] = df['Arrival_Time'].apply(lambda x: x.time())

In [12]:
# Converting Duration from hours and minutes format to integer with all entries as minutes
def extract_time(x):
    if 'h' in x and 'm' not in x:
        h = x.strip('h')
        h = h.strip(' ')
        x = (int(h)*60)
    elif 'm' in x and 'h' not in x:
        m = x.strip('m')
        m = m.strip(' ')
        x = int(m)
    elif 'h' and 'm' in x:
        h , m = x.split('h')
        h = h.strip(' ')
        m = m.strip(' ')
        m = m.strip('m')
        x = (int(h)*60) + int(m)
    return x

In [13]:
df['Duration'] = df['Duration'].apply(extract_time)

In [14]:
# Encoding the layovers
df['Total_Stops'] = df['Total_Stops'].replace({'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4})

In [15]:
# Encoding the time to categories
def time_category(x):
    if x.hour in range(0,6):
        return('early morning')
    elif x.hour in range(6,12):
        return('morning')
    elif x.hour in range(12,17):
        return('afternoon')
    elif x.hour in range(17,24):
        return('evening')

In [16]:
# Encoding the Departure Time
df['Dep_Time'] = df['Dep_Time'].apply(time_category)
# Encoding the Arrival Time
df['Arrival_Time'] = df['Arrival_Time'].apply(time_category)

In [17]:
df['Additional_Info'].replace({'No Info': 'No info'}, inplace = True)
df['Dep_Date_Month'] = df['Date_of_Journey'].apply(lambda x: x.month)
df['Arr_Date_Month'] = df['Date_of_Arrival'].apply(lambda x: x.month)
df['Dep_Date_Day'] = df['Date_of_Journey'].apply(lambda x: x.day)
df['Arr_Date_Day'] = df['Date_of_Arrival'].apply(lambda x: x.day)
df['Date_of_Journey_Weekday'] = df['Date_of_Journey'].apply(lambda x: x.weekday())
df['Date_of_Arrival_Weekday'] = df['Date_of_Arrival'].apply(lambda x: x.weekday())

#### Introducting new features to enhance the machine's perception

In [18]:
# Creating a column for the Class of travel
r = df.Airline.str.split(' ',expand=True).get(2)
d = {None:'Economy','Premium': 'Premium Economy','economy':'Premium Economy','Business':'Business' }
r = r.map(d)
df['Class']=r
df.Class=df.Class.astype('category')

In [19]:
start_date = datetime.date(2019, 1, 3) # Beginning of the month of March as the booking day
def extract_duration_before_journey(timestamp):
    delta = timestamp.date() - start_date
    return delta.days

In [20]:
df['Booking_Date'] = df['Date_of_Journey'].apply(extract_duration_before_journey)

In [21]:
# Encoding Categorical Columns
codex = LabelEncoder()
df['Class'] = codex.fit_transform(df['Class'])
df['Airline'] = codex.fit_transform(df['Airline'])
df['Additional_Info'] = codex.fit_transform(df['Additional_Info'])
df['Dep_Time'] = codex.fit_transform(df['Dep_Time'])
df['Arrival_Time'] = codex.fit_transform(df['Arrival_Time'])
df['Route'] = codex.fit_transform(df['Route'])

In [22]:
# Normalizing the price variable
df['Price'] = np.log(df['Price'])

In [23]:
df.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date_of_Arrival,Dep_Date_Month,Arr_Date_Month,Dep_Date_Day,Arr_Date_Day,Date_of_Journey_Weekday,Date_of_Arrival_Weekday,Class,Booking_Date
0,3,2019-03-24,0,2,18,2,1,170,0,7,8.267962,2019-03-22,3,3,24,22,6,4,1,80
1,1,2019-01-05,1,0,84,1,0,445,2,7,8.944028,2019-07-23,1,7,5,23,5,1,1,2
2,4,2019-09-06,2,5,118,3,1,1140,2,7,9.538348,2019-06-10,9,6,6,10,4,0,1,246
3,3,2019-12-05,1,0,91,2,2,325,1,7,8.735204,2019-07-23,12,7,5,23,3,1,1,336
4,3,2019-01-03,0,2,29,0,2,285,1,7,9.495670,2019-07-23,1,7,3,23,3,1,1,0
5,8,2019-06-24,1,0,64,3,3,145,0,7,8.261785,2019-07-23,6,7,24,23,0,1,1,172
6,4,2019-12-03,0,2,5,2,3,930,1,5,9.313529,2019-03-13,12,3,3,13,1,2,1,334
7,4,2019-01-03,0,2,5,3,1,1265,1,7,10.010996,2019-03-02,1,3,3,2,3,5,1,0
8,4,2019-12-03,0,2,5,3,3,1530,1,5,9.313529,2019-03-13,12,3,3,13,1,2,1,334
9,6,2019-05-27,2,5,104,3,2,470,1,7,9.062420,2019-07-23,5,7,27,23,0,1,1,144


#### Model Design

In [24]:
df = df.drop(['Date_of_Journey', 'Date_of_Arrival'], axis = 1) # After their sub division, the columns are rendered obsolete
x = df.columns.tolist()
y = x.pop(x.index('Price'))
x.append(y)

In [25]:
# Rearrangement of the columns
train = df[x]

In [26]:
X = train.iloc[:,:-1]
y = train['Price']
X = np.asarray(X)
y = np.asarray(y)

In [27]:
X_train, X_eval, y_train, y_eval = train_test_split(X,y, test_size = 0.2, shuffle = True, random_state = 50)

In [30]:
xgb = XGBRegressor(n_estimators = 3000, learning_rate = 0.1, gamma = 0.2, subsample = 0.8,
                    colsample_bytree = 0.8, max_depth = 25, eta = 0.1, seed = 10,
                    reg_alpha = 1e-3, objective = 'reg:squarederror', random_state = 50,
                    scale_pos_weight = 1, min_child_weight = 1, eval_metric = 'rmse')

xgb.fit(X_train,y_train)
xgb.score(X_eval,y_eval)

0.9391480807618675

In [31]:
# Fitting the model on whole data
xgb = XGBRegressor(n_estimators = 3000, learning_rate = 0.1, gamma = 0.2, subsample = 0.8,
                    colsample_bytree = 0.8, max_depth = 25, eta = 0.1, seed = 10,
                    reg_alpha = 1e-3, objective = 'reg:squarederror', random_state = 50,
                    scale_pos_weight = 1, min_child_weight = 1, eval_metric = 'rmse')
xgb.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, eta=0.1,
       eval_metric='rmse', gamma=0.2, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=25,
       min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
       nthread=None, objective='reg:squarederror', random_state=50,
       reg_alpha=0.001, reg_lambda=1, scale_pos_weight=1, seed=10,
       silent=None, subsample=0.8, verbosity=1)

#### Prediction on the test data

In [33]:
test = pd.read_excel('Test_set.xlsx')

In [34]:
test.sample(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
1132,Jet Airways,9/05/2019,Delhi,Cochin,DEL → COK,11:00,14:15,3h 15m,non-stop,In-flight meal not included
2180,Jet Airways,12/04/2019,Banglore,Delhi,BLR → DEL,15:15,18:10,2h 55m,non-stop,In-flight meal not included
2126,Jet Airways,12/06/2019,Delhi,Cochin,DEL → IDR → BOM → COK,21:25,12:35 13 Jun,15h 10m,2 stops,No info
347,IndiGo,1/04/2019,Kolkata,Banglore,CCU → HYD → BLR,15:15,20:05,4h 50m,1 stop,No info
870,Air Asia,6/05/2019,Kolkata,Banglore,CCU → IXR → DEL → BLR,15:10,23:30,8h 20m,2 stops,No info
20,IndiGo,6/03/2019,Delhi,Cochin,DEL → BOM → COK,10:45,01:35 07 Mar,14h 50m,1 stop,No info
273,Air Asia,15/05/2019,Delhi,Cochin,DEL → BLR → COK,07:55,22:25,14h 30m,1 stop,No info
854,SpiceJet,1/04/2019,Kolkata,Banglore,CCU → IXB → BLR,11:15,18:30,7h 15m,1 stop,No info
2359,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → BHO → DEL,06:45,23:25 02 Mar,40h 40m,2 stops,No info
1407,IndiGo,18/05/2019,Mumbai,Hyderabad,BOM → HYD,02:35,04:05,1h 30m,non-stop,No info


In [35]:
df = test
# Building consistency in the cities comprised in Source and Destination
df['Destination'] = df['Destination'].str.replace('New ','')
df['Destination'] = df['Destination'].str.replace(' ','')
df['Destination'] = df['Destination'].str.replace('Delhi','New Delhi')
df['Source'] = df['Source'].str.replace('Delhi','New Delhi')

In [36]:
# Encoding the cities
df['Source'] = df['Source'].replace({'Banglore': 0, 'Kolkata': 1, 'New Delhi': 2, 'Chennai': 3, 'Mumbai': 4})
df['Destination'] = df['Destination'].replace({'Banglore': 0, 'Kolkata': 1, 'New Delhi': 2, 'Cochin': 5, 'Hyderabad': 6})

In [37]:
df['Date_of_Journey'] = df['Date_of_Journey'].astype('datetime64')
df['Dep_Time'] = df['Dep_Time'].astype('datetime64[ns]')
df['Arrival_Time'] = df['Arrival_Time'].astype('datetime64[ns]')

In [38]:
# Extracting the time of departure of the flights
df['Dep_Time'] =  df['Dep_Time'].apply(lambda x: x.time())
# Extracting Arrival Time and Date
df['Date_of_Arrival'] = df['Arrival_Time'].apply(lambda x: x.date())
df['Arrival_Time'] = df['Arrival_Time'].apply(lambda x: x.time())

In [39]:
df['Duration'] = df['Duration'].apply(extract_time)

In [40]:
# Encoding the layovers
df['Total_Stops'] = df['Total_Stops'].replace({'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4})

In [41]:
# Encoding the Departure Time
df['Dep_Time'] = df['Dep_Time'].apply(time_category)
# Encoding the Arrival Time
df['Arrival_Time'] = df['Arrival_Time'].apply(time_category)

In [42]:
df['Additional_Info'].replace({'No Info': 'No info'}, inplace = True)
df['Dep_Date_Month'] = df['Date_of_Journey'].apply(lambda x: x.month)
df['Arr_Date_Month'] = df['Date_of_Arrival'].apply(lambda x: x.month)
df['Dep_Date_Day'] = df['Date_of_Journey'].apply(lambda x: x.day)
df['Arr_Date_Day'] = df['Date_of_Arrival'].apply(lambda x: x.day)
df['Date_of_Journey_Weekday'] = df['Date_of_Journey'].apply(lambda x: x.weekday())
df['Date_of_Arrival_Weekday'] = df['Date_of_Arrival'].apply(lambda x: x.weekday())

In [43]:
# Creating a column for the Class of travel
r = df.Airline.str.split(' ',expand=True).get(2)
d = {None:'Economy','Premium': 'Premium Economy','economy':'Premium Economy','Business':'Business' }
r = r.map(d)
df['Class']=r
df.Class=df.Class.astype('category')

In [44]:
df['Booking_Date'] = df['Date_of_Journey'].apply(extract_duration_before_journey)

In [45]:
df['Class'] = codex.fit_transform(df['Class'])
df['Airline'] = codex.fit_transform(df['Airline'])
df['Additional_Info'] = codex.fit_transform(df['Additional_Info'])
df['Dep_Time'] = codex.fit_transform(df['Dep_Time'])
df['Arrival_Time'] = codex.fit_transform(df['Arrival_Time'])
df['Route'] = codex.fit_transform(df['Route'])

In [46]:
df = df.drop(['Date_of_Journey', 'Date_of_Arrival'], axis = 1) # After their sub division, the columns are rendered obsolete
x = df.columns.tolist()
test_data = df[x]
test_data = np.asarray(test_data)

In [47]:
price_pred = xgb.predict(test_data)

In [54]:
# Taking reverse log to get actual price
#price = np.exp(price_pred)
price = (np.around(price)).astype(int)

In [55]:
print(price)

[ 9968  5374 14344 ... 14196 10946  7204]


In [56]:
# Saving the price to an excel file
df = pd.DataFrame({'Price':price})
df.to_excel('submission.xlsx', index = False)